In [1]:
from src.api import DataAPI , Predictor

DataAPI.update()
DataAPI.prepare_predict_data()
Predictor.update_factors()

Update Files
Sun Sep 15 22:55:15 2024 : Updated ~ DB_information\calendar.feather Done! Cost 0.02 Secs
Sun Sep 15 22:55:15 2024 : Updated ~ DB_information\description.feather Done! Cost 0.03 Secs
Sun Sep 15 22:55:15 2024 : Updated ~ DB_information\st.feather Done! Cost 0.03 Secs
Sun Sep 15 22:55:17 2024 : Updated ~ DB_information\industry.feather Done! Cost 2.57 Secs
Sun Sep 15 22:55:22 2024 : Updated ~ DB_information\concepts.feather Done! Cost 5.15 Secs
Sun Sep 15 22:55:59 2024 : Updated ~ DB_benchmark\csi300\2024\csi300.20240909.feather Done! Cost 0.00 Secs
Sun Sep 15 22:55:59 2024 : Updated ~ DB_benchmark\csi500\2024\csi500.20240909.feather Done! Cost 0.01 Secs
Sun Sep 15 22:55:59 2024 : Updated ~ DB_benchmark\csi1000\2024\csi1000.20240909.feather Done! Cost 0.00 Secs
Sun Sep 15 22:55:59 2024 : Updated ~ DB_benchmark\csi300\2024\csi300.20240910.feather Done! Cost 0.02 Secs
Sun Sep 15 22:55:59 2024 : Updated ~ DB_benchmark\csi500\2024\csi500.20240910.feather Done! Cost 0.00 Secs
Sun

Predictor(reg_model=RegModel(name=gru_avg,type=swabest,num=all,alias=gru_day_V1))

In [8]:
from src.data.tushare.download.info import Calendar , pro
renamer = {'cal_date' : 'calendar' ,'is_open'  : 'trade'}
fields = None
df = pro.trade_cal(exchange='SSE').rename(columns=renamer)
#df = df.sort_values('calendar').reset_index(drop = True)
df

""


In [10]:
# DataAPI.prepare_predict_data()
Predictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


In [1]:
from src.api import DataAPI , Predictor
# DataAPI.prepare_predict_data()
md = Predictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


In [ ]:
from src.api import ModelTestor
ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
